# Plot yeast RBD DMS escape maps

## Import modules and read data
Import Python modules:

In [1]:
import itertools

import altair as alt

import numpy

import pandas as pd

import sklearn.manifold

Disable max rows specifier for Altair:

In [2]:
_ = alt.data_transformers.disable_max_rows()

Read the deep mutational scanning data, and reduce to site-level data, calculating the max, mean, and total site-based metrics:

In [3]:
dms_mut_data = pd.read_csv('./results/merged_data/yeast_RBD_DMS_data.csv')

# calculate site metrics and fill missing sites as 0
sites = list(range(dms_mut_data['site'].min(), dms_mut_data['site'].max() + 1))
dms_data = (
    dms_mut_data
    .groupby(['condition', 'condition_type', 'condition_subtype', 'study', 'site'],
             as_index=False, dropna=False)
    .aggregate(site_total_escape=pd.NamedAgg('mut_escape', 'sum'),
               site_max_escape=pd.NamedAgg('mut_escape', 'max'),
               site_mean_escape=pd.NamedAgg('mut_escape', 'mean')
               )
    )
assert dms_data.notnull().all().all()
dms_data = (pd.merge_ordered(dms_data,
                             pd.DataFrame({'site': sites}),
                             on='site',
                             left_by=['condition', 'study', 'condition_type', 'condition_subtype'],
                             )
            .fillna(0)
            )

# check no duplicated conditions
dup_conditions = (dms_data
                  .groupby('condition', as_index=False)
                  .aggregate(n_studies=pd.NamedAgg('study', 'nunique'))
                  .query('n_studies > 1')
                  )
if len(dup_conditions):
    raise ValueError('duplicate studies for some conditions:\n' + str(dup_conditions))

dms_data

,condition,condition_type,condition_subtype,study,site,site_total_escape,site_max_escape,site_mean_escape
0,C002,antibody,not clinical antibody,2021_Greaney_Rockefeller,331,0.028500,0.001785,0.001781
1,C002,antibody,not clinical antibody,2021_Greaney_Rockefeller,332,0.033839,0.001781,0.001781
2,C002,antibody,not clinical antibody,2021_Greaney_Rockefeller,333,0.032058,0.001781,0.001781
3,C002,antibody,not clinical antibody,2021_Greaney_Rockefeller,334,0.032058,0.001781,0.001781
4,C002,antibody,not clinical antibody,2021_Greaney_Rockefeller,335,0.033839,0.001781,0.001781
...,...,...,...,...,...,...,...,...
9844,subject K (day 29),serum,convalescent serum,2021_Greaney_HAARVI_sera,527,0.005310,0.002243,0.000312
9845,subject K (day 29),serum,convalescent serum,2021_Greaney_HAARVI_sera,528,0.009810,0.002504,0.000545
9846,subject K (day 29),serum,convalescent serum,2021_Greaney_HAARVI_sera,529,0.031831,0.009543,0.001768
9847,subject K (day 29),serum,convalescent serum,2021_Greaney_HAARVI_sera,530,0.024201,0.007600,0.001274


Make a tidy version of `dms_data` that is melted to have the two site metrics in one column, and gets rid of some columns we don't need for escape line plots:

In [4]:
tidy_cols = {'site_total_escape': 'sum of mutations at site',
             'site_max_escape': 'max of any mutation at site',
             'site_mean_escape': 'mean of mutations at site'}
dms_data_tidy = (
    dms_data
    .rename(columns=tidy_cols)
    .melt(value_vars=tidy_cols.values(),
          value_name='escape',
          var_name='metric',
          id_vars=[c for c in dms_data.columns if c not in tidy_cols])
    .drop(columns=['condition_type', 'study'])
    )

dms_data_tidy

,condition,condition_subtype,site,metric,escape
0,C002,not clinical antibody,331,sum of mutations at site,0.028500
1,C002,not clinical antibody,332,sum of mutations at site,0.033839
2,C002,not clinical antibody,333,sum of mutations at site,0.032058
3,C002,not clinical antibody,334,sum of mutations at site,0.032058
4,C002,not clinical antibody,335,sum of mutations at site,0.033839
...,...,...,...,...,...
29542,subject K (day 29),convalescent serum,527,mean of mutations at site,0.000312
29543,subject K (day 29),convalescent serum,528,mean of mutations at site,0.000545
29544,subject K (day 29),convalescent serum,529,mean of mutations at site,0.001768
29545,subject K (day 29),convalescent serum,530,mean of mutations at site,0.001274


## Perform multidimensional scaling
Steps:
 1. Calculate similarities betweeen escape maps for each antibody.
 2. Convert similarities to dissimilarities.
 3. Do multi-dimensional scaling on dissimilarities.


First, compute the dissimilarity between all pairs of escape profiles in a data frame.
We calculate similarity as the dot product of the escape profile site-level metric for each pair of conditions, normalizing each profile so it's dot product with itself is one.
Then we compute the dissimilarity as just one minux the similarity:

In [5]:
def escape_similarity(df):
    """Compute similarity between all pairs of conditions in `df`."""
    df = df[['condition', 'site', 'escape']].drop_duplicates()
    assert not df.isnull().any().any()
    
    conditions = df['condition'].unique()
    similarities = []
    pivoted_df = (
        df
        .pivot_table(index='site',
                     columns='condition',
                     values='escape',
                     fill_value=0)
        # for normalization: https://stackoverflow.com/a/58113206
        # to get norm: https://stackoverflow.com/a/47953601
        .transform(lambda x: x / numpy.linalg.norm(x, axis=0))
        )
    for cond1, cond2 in itertools.product(conditions, conditions):
        similarity = (
            pivoted_df
            [list({cond1, cond2})]
            .assign(similarity=lambda x: x[cond1] * x[cond2])
            ['similarity']
            )
        assert similarity.notnull().all()  # make sure no sites have null values
        similarities.append(similarity.sum())  # sum of similarities over sites
    return pd.DataFrame(numpy.array(similarities).reshape(len(conditions), len(conditions)),
                        columns=conditions, index=conditions)

similarities = (
    dms_data_tidy
    .groupby('metric')
    .apply(escape_similarity)
    )

dissimilarities = (1 - similarities).clip(lower=0)

dissimilarities.round(3)

C002   C105   C110   C121  \
metric                                                                        
max of any mutation at site C002                 0.000  0.731  0.736  0.192   
                            C105                 0.731  0.000  0.991  0.814   
                            C110                 0.736  0.991  0.000  0.722   
                            C121                 0.192  0.814  0.722  0.000   
                            C135                 0.987  0.985  0.480  0.972   
...                                                ...    ...    ...    ...   
sum of mutations at site    subject I (day 26)   0.206  0.733  0.705  0.301   
                            subject J (day 121)  0.607  0.693  0.546  0.717   
                            subject J (day 15)   0.695  0.777  0.792  0.771   
                            subject K (day 103)  0.509  0.671  0.870  0.626   
                            subject K (day 29)   0.690  0.799  0.857  0.746   

                                                  C135   C144  COV-021  \
metric                                                                   
max of any mutation at site C002                 0.987  0.132    0.352   
                            C105                 0.985  0.696    0.715   
                            C110                 0.480  0.764    0.739   
                            C121                 0.972  0.084    0.326   
                            C135                 0.000  0.985    0.823   
...                                                ...    ...      ...   
sum of mutations at site    subject I (day 26)   0.934  0.295    0.148   
                            subject J (day 121)  0.784  0.659    0.362   
                            subject J (day 15)   0.915  0.686    0.508   
                            subject K (day 103)  0.928  0.423    0.373   
                            subject K (day 29)   0.944  0.679    0.523   

                                                 COV-047  COV-057  COV-072  \
metric                                                                       
max of any mutation at site C002                   0.374    0.558    0.537   
                            C105                   0.669    0.770    0.668   
                            C110                   0.623    0.596    0.716   
                            C121                   0.314    0.522    0.537   
                            C135                   0.731    0.679    0.756   
...                                                  ...      ...      ...   
sum of mutations at site    subject I (day 26)     0.224    0.299    0.264   
                            subject J (day 121)    0.477    0.572    0.305   
                            subject J (day 15)     0.500    0.584    0.334   
                            subject K (day 103)    0.168    0.248    0.070   
                            subject K (day 29)     0.475    0.555    0.308   

                                                 ...  subject G (day 18)  \
metric                                           ...                       
max of any mutation at site C002                 ...               0.692   
                            C105                 ...               0.855   
                            C110                 ...               0.392   
                            C121                 ...               0.671   
                            C135                 ...               0.534   
...                                              ...                 ...   
sum of mutations at site    subject I (day 26)   ...               0.464   
                            subject J (day 121)  ...               0.386   
                            subject J (day 15)   ...               0.612   
                            subject K (day 103)  ...               0.608   
                            subject K (day 29)   ...               0.669   

                                                 subjec

Now do the multidimensional scaling [as described here](https://scikit-learn.org/stable/auto_examples/manifold/plot_mds.html#sphx-glr-auto-examples-manifold-plot-mds-py) to get the x and y coordinates for each antibody / serum.
For each metric, we do this for three different random number seeds (different seeds will given different MDS layouts):

In [6]:
mds_coords = []
for seed, (metric, mat) in itertools.product([1, 2], dissimilarities.groupby('metric')):
    # use multidimensional scaling to get locations of antibodies
    mds = sklearn.manifold.MDS(n_components=2,
                               metric=True,
                               max_iter=3000,
                               eps=1e-6,
                               random_state=seed,
                               dissimilarity='precomputed',
                               n_jobs=1)
    locs = mds.fit_transform(mat)
    mds_coords.append(pd.DataFrame(locs, columns=['x', 'y'])
                      .assign(metric=metric,
                              seed=seed,
                              condition=mat.columns,
                              xmin=lambda df: df['x'].min(),
                              ymin=lambda df: df['y'].min(),
                              x=lambda df: df['x'] - df['xmin'],
                              y=lambda df: df['y'] - df['ymin'],
                              )
                      )
mds_coords = (
    pd.concat(mds_coords,
              ignore_index=True)
    .merge(dms_data
           [['condition', 'condition_type', 'condition_subtype', 'study']]
           .drop_duplicates(),
           on='condition',
           how='left',
           validate='many_to_one')
    .drop(columns=['xmin', 'ymin'])
    )
mds_coords

,x,y,metric,seed,condition,condition_type,condition_subtype,study
0,0.855700,1.164560,max of any mutation at site,1,C002,antibody,not clinical antibody,2021_Greaney_Rockefeller
1,1.238307,0.489368,max of any mutation at site,1,C105,antibody,not clinical antibody,2021_Greaney_Rockefeller
2,0.142164,1.059232,max of any mutation at site,1,C110,antibody,not clinical antibody,2021_Greaney_Rockefeller
3,0.821894,1.190007,max of any mutation at site,1,C121,antibody,not clinical antibody,2021_Greaney_Rockefeller
4,0.020785,0.571831,max of any mutation at site,1,C135,antibody,not clinical antibody,2021_Greaney_Rockefeller
...,...,...,...,...,...,...,...,...
289,0.891555,0.805348,sum of mutations at site,2,subject I (day 26),serum,convalescent serum,2021_Greaney_HAARVI_sera
290,0.556415,0.673534,sum of mutations at site,2,subject J (day 121),serum,convalescent serum,2021_Greaney_HAARVI_sera
291,0.526010,0.495953,sum of mutations at site,2,subject J (day 15),serum,convalescent serum,2021_Greaney_HAARVI_sera
292,0.852723,0.504828,sum of mutations at site,2,subject K (day 103),serum,convalescent serum,2021_Greaney_HAARVI_sera


## Make interactive plots
First make plot to select condition(s) both to show:

In [7]:
conditions_df = (
    mds_coords
    [['condition_type', 'condition_subtype', 'condition']]
    .sort_values(['condition_type', 'condition_subtype', 'condition'])
    .drop_duplicates()
    )

condition_subtypes = (conditions_df
                      ['condition_subtype']
                      .unique()
                      .tolist()
                      )

# define colors from here: https://vega.github.io/vega/docs/schemes/
condition_subtype_colors = {'clinical antibody': '#0072B2',
                            'not clinical antibody': '#56B4E9',
                            'convalescent serum': '#FD5602',
                            'Moderna serum': '#FFAF42',
                            }
if not set(condition_subtypes).issubset(condition_subtype_colors):
    raise ValueError('missing colors for some condition subtypes')
select_condition_subtype = alt.selection_multi(fields=['condition_subtype'],
                                               # initialize to show antibodies but not sera
                                               init=[{'condition_subtype': subtype} for subtype in
                                                     conditions_df.query('condition_type == "antibody"')
                                                     ['condition_subtype'].unique()],
                                               resolve='union',
                                               empty='none',
                                               )
condition_subtype_color = alt.condition(select_condition_subtype,
                                   alt.Color('condition_subtype:N',
                                             legend=None,
                                             scale=alt.Scale(domain=condition_subtypes,
                                                             range=[condition_subtype_colors[c]
                                                                    for c in condition_subtypes]),
                                                             ),
                                   alt.value('white'),
                                   )

circle_size = 110

legend_condition_type = (
    alt.Chart(conditions_df[['condition_type', 'condition_subtype']].drop_duplicates())
    .mark_circle(size=0.7 * circle_size,
                 stroke='black',
                 strokeWidth=1)
    .encode(x=alt.X('condition_type:N',
                    axis=alt.Axis(title=['',
                                         'On each subplot, you can:',
                                         ' - click to select one item',
                                         ' - shift-click to select additional items',
                                         ' - double-click to clear selections',
                                         ' - mouseover to see antibody / serum name',
                                         ],
                                  titleAlign='left',
                                  titleFontSize=14,
                                  titleFontWeight='normal',
                                  titleFontStyle='italic',
                                  labelFontSize=12),
                    ),
            y=alt.Y('condition_subtype:N',
                    sort=condition_subtypes,
                    axis=alt.Axis(title=None,
                                  labelFontSize=12),
                    ),
            color=condition_subtype_color,
            )
    .add_selection(select_condition_subtype)
    .properties(title={'text': ['choose antibody/serum',
                                'types to display'],
                       'align': 'left',
                       'anchor': 'start'})
    )

legend_condition_type.configure_view(strokeOpacity=0)

alt.Chart(...)

In [8]:
highlight_condition = (
    alt.selection(type='multi',
                  on='click',
                  fields=['condition'],
                  nearest=False,
                  empty='none',
                  toggle=True,
                  resolve='union',
                  )
    )

# build zoom bar to zoom in condition legend
legend_condition_zoom_brush = alt.selection_interval(
                encodings=['y'],
                mark=alt.BrushConfig(stroke='black', strokeWidth=2))
legend_condition_zoom_bar = (
    alt.Chart(conditions_df)
    .mark_rect()
    .encode(y=alt.Y('condition:N',
                    title='antibody / sera zoom bar',
                    sort=conditions_df['condition'].unique(),
                    axis=alt.Axis(ticks=False,
                                  labels=False,
                                  titleFontSize=12)
                    ),
            color=condition_subtype_color,
            )
    .add_selection(legend_condition_zoom_brush)
    .transform_filter(select_condition_subtype)
    .properties(height=150,
                width=15)
    )

legend_condition_heatmap = (
    alt.Chart(conditions_df)
    .encode(y=alt.Y('condition:N',
                    sort=conditions_df['condition'].unique(),
                    title=None,
                    axis=alt.Axis(orient='right',
                                  labelFontSize=11,
                                  ),
                    ),
            color=condition_subtype_color,
            strokeWidth=alt.condition(~highlight_condition,
                                      alt.value(0.5),
                                      alt.value(3)),
            stroke=alt.condition(~highlight_condition,
                                 alt.value('black'),
                                 alt.value('black')),
            )
    .mark_rect()
    .add_selection(select_condition_subtype,
                   highlight_condition)
    .transform_filter(select_condition_subtype)
    .transform_filter(legend_condition_zoom_brush)
    .properties(height={'step': 15},
                width=15,
                )
    )

condition_citations = (
    alt.Chart(conditions_df)
    .encode(y=alt.Y('condition:N',
                    sort=conditions_df['condition'].unique(),
                    title=None,
                    axis=None,
                    ),
            text='condition:N',
            )
    .mark_text(align='left',
               fontSize=11,
               fontStyle='normal',
               href='http://www.google.com',
               )
    .add_selection(select_condition_subtype,
                   highlight_condition)
    .transform_filter(select_condition_subtype)
    .transform_filter(legend_condition_zoom_brush)
    .properties(height={'step': 15},
                width=15,
                )
    )

legend_condition = (
    (legend_condition_zoom_bar | alt.hconcat(legend_condition_heatmap,
                                             condition_citations,
                                             spacing=2)
     )
    .properties(title={'text': ['choose antibodies/sera by name by clicking box;',
                                'shift-click citation or dms-view text to open that information']})
    )

legend_condition.configure_view(strokeOpacity=0)

alt.HConcatChart(...)

Next make MDS plot:

In [9]:
# build drop down menu to select metric and random seed
metric_select_binding = alt.binding_select(options=mds_coords['metric'].unique())
metric_selection = alt.selection_single(name='escape',
                                        fields=['metric'],
                                        bind=metric_select_binding,
                                        init={'metric': 'sum of mutations at site'})
seed_select_binding = alt.binding_select(options=mds_coords['seed'].unique())
seed_selection = alt.selection_single(name='multidimensional scaling random',
                                      fields=['seed'],
                                      bind=seed_select_binding,
                                      init={'seed': 1},
                                      )

# size, but scaled so a unit on x and y mean the same; note
# padding added here so sizes correct
size = 180
pad = 0.04
x_extent = mds_coords['x'].max() - mds_coords['x'].min()
y_extent = mds_coords['y'].max() - mds_coords['y'].min()
y_min = mds_coords['y'].min() - pad * y_extent
y_max = mds_coords['y'].max() + pad * y_extent
x_min = mds_coords['x'].min() - pad * x_extent
x_max = mds_coords['x'].max() + pad * x_extent

mds_plot = (
    alt.Chart(mds_coords)
    .encode(x=alt.X('x:Q',
                    scale=alt.Scale(padding=0,
                                    nice=False,
                                    domain=(x_min, x_max),
                                    ),
                    axis=alt.Axis(labels=False,
                                  title=None,
                                  ticks=False,
                                  grid=False,
                                  ),
                    ),
            y=alt.Y('y:Q',
                    scale=alt.Scale(padding=0,
                                    nice=False,
                                    domain=(y_min, y_max),
                                    ),
                    axis=alt.Axis(labels=False,
                                  title=None,
                                  ticks=False,
                                  grid=False,
                                  ),
                    ),
            opacity=alt.condition(~highlight_condition, alt.value(0.75), alt.value(1)),
            stroke=alt.condition(~highlight_condition, alt.value(None), alt.value('black')),
            color=condition_subtype_color,
            tooltip=['condition'])
    .mark_circle(size=circle_size)
    .properties(width=size * x_extent,
                height=size * y_extent,
                title={'text': 'multidimensional scaling of antibodies/sera',
                       'subtitle': ['antibodies/sera with escape mutations at similar',
                                    'sites are positioned nearby in the plot below'],
                       'anchor': 'start',
                       'align': 'left',
                       }
                )
    .add_selection(seed_selection,
                   metric_selection,
                   highlight_condition,
                   select_condition_subtype,
                   )
    .transform_filter(metric_selection)
    .transform_filter(seed_selection)
    .transform_filter(select_condition_subtype)
    )

# box around MDS plot: https://stackoverflow.com/a/62862229/4191652
dummy_lines = {}
for key, x, y in [('top', (x_min, x_max), (y_max, y_max)),
                  ('right', (x_max, x_max), (y_min, y_max)),
                  ]:
    dummy_lines[key] = (
        alt.Chart(pd.DataFrame({'x': x,
                                'y': y})
                  )
        .mark_line(color='black',
                   strokeWidth=0.5)
        .encode(x=alt.X('x:Q',
                        scale=alt.Scale(padding=0,
                                        nice=False,
                                        domain=(x_min, x_max),
                                        ),
                        axis=alt.Axis(labels=False,
                                      title=None,
                                      ticks=False,
                                      grid=False,
                                      ),
                        ),
                y=alt.Y('y:Q',
                        scale=alt.Scale(padding=0,
                                        nice=False,
                                        domain=(y_min, y_max),
                                        ),
                        axis=alt.Axis(labels=False,
                                      title=None,
                                      ticks=False,
                                      grid=False,
                                      ),
                        )
                )
        )
mds_plot = mds_plot + dummy_lines['top'] + dummy_lines['right']

# show the plot with legend
(legend_condition_type | mds_plot).configure_view(stroke='black').configure_view(strokeOpacity=0)

alt.HConcatChart(...)

Next make line plots:

In [ ]:
width = 800

# build zoom bar to zoom in on sites
zoom_brush = alt.selection_interval(
                encodings=['x'],
                mark=alt.BrushConfig(stroke='black', strokeWidth=2))
zoom_bar = (
    alt.Chart(dms_data_tidy[['site']].drop_duplicates())
    .mark_rect(color='lightgray')
    .encode(x=alt.X('site:O',
                    title=None,
                    ),
            )
    .add_selection(zoom_brush)
    .properties(width=width,
                height=15,
                title='site zoom bar')
    )

# build base for escape plots
escape_base = (
    alt.Chart(dms_data_tidy.assign(all_antibodies_sera_of_displayed_types=True))
    .encode(x=alt.X('site:O',
                    axis=alt.Axis(grid=False),
                    ),
            )
    .transform_filter(metric_selection)
    .transform_filter(select_condition_subtype)
    .transform_filter(zoom_brush)
    .properties(width=width,
                height=200,
                )
    )

# the escape line plot
escape_lines = (
    escape_base
    .encode(size=alt.condition(~highlight_condition, alt.value(0.9), alt.value(1.5)),
            opacity=alt.condition(~highlight_condition, alt.value(0.4), alt.value(1)),
            )
    .add_selection(metric_selection,
                   select_condition_subtype,
                   zoom_brush,
                   )
    .mark_line()
    )

# escape point plot
escape_points = (
    escape_base
    .encode(fill=condition_subtype_color,
            tooltip=['condition:N', 'site:O'],
            )
    .mark_point(size=40)
    .transform_filter(highlight_condition)
    # needs to be add_selection within chart: https://github.com/altair-viz/altair/issues/2368#issuecomment-742377146
    .add_selection(highlight_condition)
    )

# combine point and line plots
escape_lines_points = (
    (escape_lines + escape_points)
    .encode(detail='condition:N',  # https://github.com/altair-viz/altair/issues/985
            color=condition_subtype_color,
            y=alt.Y('escape:Q',
                    axis=alt.Axis(grid=False),
                    ),
            )
    .properties(title={'text': 'escape from individual antibodies/sera'})
    )

# checkbox to specify if mean for only selected antibodies or all antibody/serum types
mean_radio = alt.binding_radio(options=[True, False])
mean_selection = alt.selection_single(fields=['all_antibodies_sera_of_displayed_types'],
                                               bind=mean_radio,
                                               name='mean_over',
                                               init={'all_antibodies_sera_of_displayed_types': False})
# plot of mean values
escape_mean = (
    escape_base
    .mark_line(color='darkgray',
               point={'color': 'darkgray',
                      'size': 60},
               )
    .encode(tooltip=['site:O'],
            y=alt.Y('mean(escape):Q',
                    axis=alt.Axis(grid=False,
                                  title='escape',
                                  ),
                    ),
            )
    .transform_filter(highlight_condition | (select_condition_subtype & mean_selection))
    .add_selection(highlight_condition,
                   mean_selection,
                   )
    .properties(title={'text': 'mean escape over selected antibodies/sera or ' +
                               'all antibodies/sera of displayed types (see ' +
                               'radio button selection below)',
                       })
    )

# combine zoom bar, lines, and points
escape_plot = (zoom_bar & (escape_lines_points) & escape_mean).resolve_scale(x='shared')

escape_plot

Now combine the antibody MDS and escape plots:

In [ ]:
chart = (
    (((mds_plot | legend_condition_type) & escape_plot) | legend_condition)
    .configure(padding={'left': 5,
                        'right': 50,
                        'top': 5,
                        'bottom': 5})
    .configure_view(strokeOpacity=0)
    )
chart.save('chart.html')
chart

https://github.com/altair-viz/altair/issues/1084

https://github.com/vega/vega-lite/issues/3795